In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
# Caminho do arquivo XLS
arquivo = "../Data/IBGE/RELATORIO_DTB_BRASIL_MUNICIPIO.xls"

In [3]:
!pip install xlrd

In [4]:
# Carregar o arquivo XLS ignorando as 6 primeiras linhas (começa da linha 7)
df_municipios = pd.read_excel(arquivo, skiprows=6)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [5]:
siglas_estados = {
    "Acre": "AC", "Alagoas": "AL", "Amapá": "AP", "Amazonas": "AM", "Bahia": "BA",
    "Ceará": "CE", "Distrito Federal": "DF", "Espírito Santo": "ES", "Goiás": "GO",
    "Maranhão": "MA", "Mato Grosso": "MT", "Mato Grosso do Sul": "MS", "Minas Gerais": "MG",
    "Pará": "PA", "Paraíba": "PB", "Paraná": "PR", "Pernambuco": "PE", "Piauí": "PI",
    "Rio de Janeiro": "RJ", "Rio Grande do Norte": "RN", "Rio Grande do Sul": "RS",
    "Rondônia": "RO", "Roraima": "RR", "Santa Catarina": "SC", "São Paulo": "SP",
    "Sergipe": "SE", "Tocantins": "TO"
}


In [6]:
estados_regioes = {
    'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'Centro-Oeste': ['DF', 'GO', 'MT', 'MS'],
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
    'Sul': ['PR', 'RS', 'SC']
}

In [7]:
df_municipios["UF"] = df_municipios["Nome_UF"].map(siglas_estados)


In [8]:
# 🔹 **Função para mapear estados para regiões**
def definir_regiao(uf):
    for regiao, estados in estados_regioes.items():
        if uf in estados:
            return regiao
    return 'Desconhecido'  # Caso algum estado não esteja mapeado

# 🔹 **Adicionar a nova coluna regiao**
df_municipios['regiao'] = df_municipios['UF'].apply(definir_regiao)

In [9]:
df_localizacoes = df_municipios[['Código Município Completo','Nome_Município','UF','regiao']]

In [10]:
df_localizacoes.head()

,Código Município Completo,Nome_Município,UF,regiao
0,1100015,Alta Floresta D'Oeste,RO,Norte
1,1100379,Alto Alegre dos Parecis,RO,Norte
2,1100403,Alto Paraíso,RO,Norte
3,1100346,Alvorada D'Oeste,RO,Norte
4,1100023,Ariquemes,RO,Norte


In [11]:
df_localizacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código Município Completo  5570 non-null   int64 
 1   Nome_Município             5570 non-null   object
 2   UF                         5570 non-null   object
 3   regiao                     5570 non-null   object
dtypes: int64(1), object(3)
memory usage: 174.2+ KB


In [12]:
# 🔹 **Renomear colunas para padronização**
df_localizacoes.rename(columns={
    'Código Município Completo': 'id_local',
    'Nome_Município': 'municipio',
    'UF': 'estado'
}, inplace=True)

C:\Users\geiso\AppData\Local\Temp\ipykernel_27624\2848960553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_localizacoes.rename(columns={


In [13]:
df_localizacoes.head()

,id_local,municipio,estado,regiao
0,1100015,Alta Floresta D'Oeste,RO,Norte
1,1100379,Alto Alegre dos Parecis,RO,Norte
2,1100403,Alto Paraíso,RO,Norte
3,1100346,Alvorada D'Oeste,RO,Norte
4,1100023,Ariquemes,RO,Norte


In [14]:
# Remover o último digito da direita para a esquerda
df_localizacoes.loc[:, 'id_local'] = df_localizacoes['id_local'].astype(str).str[:-1].astype(int)

In [15]:
df_localizacoes

,id_local,municipio,estado,regiao
0,110001,Alta Floresta D'Oeste,RO,Norte
1,110037,Alto Alegre dos Parecis,RO,Norte
2,110040,Alto Paraíso,RO,Norte
3,110034,Alvorada D'Oeste,RO,Norte
4,110002,Ariquemes,RO,Norte
...,...,...,...,...
5565,522200,Vianópolis,GO,Centro-Oeste
5566,522205,Vicentinópolis,GO,Centro-Oeste
5567,522220,Vila Boa,GO,Centro-Oeste
5568,522230,Vila Propício,GO,Centro-Oeste


Fazer a inserção no banco de dados

In [16]:
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mortalidades"

In [17]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [18]:
with engine.connect() as conn:
    print("✅ Conectado ao banco mortalidade")


✅ Conectado ao banco mortalidade


In [19]:
def inserir_dados(df, tabela):
    df.to_sql(tabela, con=engine, if_exists="append", index=False)
    print(f"✅ Dados inseridos na tabela {tabela}")

In [20]:
# 🔹 **Inserir os Dados no PostgreSQL**
df_localizacoes.to_sql("localizacoes", con=engine, if_exists="append", index=False)

print("✅ Dados inseridos com sucesso na tabela localizacoes!")

✅ Dados inseridos com sucesso na tabela localizacoes!
